In [4]:
import json
import random
from faker import Faker
from datetime import datetime, timedelta
from faker.providers import address

In [5]:
fake = Faker("ru_RU")

def generate_seat_types():
    seat_types = ["плацкарт", "купе", "св", "люкс"]
    num = random.randint(1, len(seat_types))
    types = random.sample(seat_types, num)
    return ' '.join(types)

def random_date(start_year, end_year):
    random_year = random.randint(start_year, end_year)
    random_day = random.randint(1, 365)
    random_hour = random.randint(0, 23)
    random_minute = random.randint(0, 59)
    return datetime(year=random_year, month=1, day=1, hour=random_hour, minute=random_minute) + timedelta(days=random_day)

def generate_train(index):
    dep_date = random_date(2010, 2020)
    train = {
        "index": "train",
        "id": index,
        "body": {
            "train_number" : fake.random_number(digits=6),
            "train_info" : generate_seat_types() + " " + dep_date.strftime("%Y-%m-%dT%H:%M") + " " + 
                (dep_date + timedelta(days=random.randint(0, 6), hours= random.randint(0, 23), minutes=random.randint(0, 59))).strftime("%Y-%m-%dT%H:%M"),
            "departure" : fake.city_name(),
            "arrive": fake.city_name(),
            "tickets_sold" : 0,
            "tickets_remain" : fake.random_int(min=100, max=1000)
        }
    }
    return train

def generate_trains(num):
    trains = []
    for i in range(num):
        trains.append(generate_train(i+1))
    return trains

def generate_tickets(num, trains):
    tickets = []
    for i in range(num):
        idx = random.randint(0, len(trains) - 1)
        trains[idx]['body']['tickets_sold'] += 1
        trains[idx]['body']['tickets_remain'] -= 1
        ticket = {
            "index": "ticket",
            "id": i+1,
            "body": {
                "user_id": fake.uuid4(),
                "personal": fake.name(),
                "date_purchase": (datetime.strptime(trains[idx]['body']['train_info'].split(" ")[-2], "%Y-%m-%dT%H:%M") 
                                  - timedelta(days=random.randint(1, 30), hours=random.randint(0, 23), seconds=random.randint(0, 59))).isoformat(),
                "price": round(random.uniform(500, 20000), 2),
                "train_id": trains[idx]['id'],
            }
        }
        tickets.append(ticket)
    return tickets

In [6]:
trains = generate_trains(20)
tickets = generate_tickets(30, trains)
with open("tickets.json", "w") as f:
    json.dump(tickets, f, indent=4, ensure_ascii=False)
with open("trains.json", "w") as f:
    json.dump(trains, f, indent=4, ensure_ascii=False)